In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Get weekly data

In [2]:
df = pd.read_csv('data/weather/order_116861_data.txt', sep=';')

df.columns
df.shape

(45659, 12)

In [3]:
# some light dusting

# Drop columns where all elements are NaN
df = df.dropna(axis=1, how='all')

# drop random reapeated header columns
header_list = df.columns.tolist()
header_rows = df[df.isin(header_list).all(axis=1)]
df = df.drop(header_rows.index)

# convert format of time column
df['time'] = pd.to_datetime(df['time'], format='mixed')


df.shape

(45648, 12)

In [4]:
# aggregate from daily to weekly data

df.set_index(['time', 'stn'], inplace=True)

# Convert columns to numeric, errors='coerce' will turn non-convertible values to NaN
for col in df.columns:
    if col not in ['time', 'stn']:  # Exclude non-numeric columns like 'time' and 'stn'
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Group by 'stn' and week number, then calculate the mean
weekly_data = df.groupby([pd.Grouper(level='stn'), pd.Grouper(level='time', freq='W')]).mean()

# Resetting the index if needed
weekly_data.reset_index(inplace=True)

In [5]:
weekly_data

,stn,time,fu3010d1,tre200dx,tre200dn,tre200d0,rka150d0,ure200dx,ure200dn,ure200d0,sre000d0,fu3010d0
0,BAS,2013-01-06,22.566667,6.916667,1.500000,4.616667,0.366667,92.816667,74.983333,84.533333,46.000000,5.500000
1,BAS,2013-01-13,17.628571,3.357143,0.100000,1.657143,2.471429,96.600000,79.242857,89.114286,44.428571,4.771429
2,BAS,2013-01-20,28.842857,-0.614286,-4.000000,-2.142857,1.842857,92.128571,69.500000,82.942857,80.142857,6.542857
3,BAS,2013-01-27,24.314286,0.857143,-2.971429,-0.828571,0.328571,93.114286,71.342857,83.200000,97.428571,5.757143
4,BAS,2013-02-03,46.185714,10.185714,2.971429,6.757143,5.857143,91.800000,56.814286,76.342857,89.000000,11.071429
...,...,...,...,...,...,...,...,...,...,...,...,...
6523,STG,2023-05-07,32.642857,16.585714,8.228571,12.157143,7.214286,95.914286,55.971429,77.242857,385.428571,6.685714
6524,STG,2023-05-14,31.171429,12.785714,8.371429,10.014286,8.757143,97.585714,74.000000,89.914286,59.857143,6.528571
6525,STG,2023-05-21,30.600000,13.771429,6.471429,9.957143,8.514286,95.414286,64.442857,82.400000,102.142857,7.185714
6526,STG,2023-05-28,27.157143,19.171429,10.171429,14.528571,1.971429,92.185714,55.042857,74.428571,458.428571,6.485714


# Group data by region 

In [6]:
df.reset_index(inplace=True)
df.columns

Index(['time', 'stn', 'fu3010d1', 'tre200dx', 'tre200dn', 'tre200d0',
       'rka150d0', 'ure200dx', 'ure200dn', 'ure200d0', 'sre000d0', 'fu3010d0'],
      dtype='object')

In [7]:
# assigning region
stn_dict = {
    'GVE': 1, 'CDF': 1, 'NEU': 1, 'SIO': 1,
    'BER': 2, 'BAS': 3, 'LUZ': 4, 'STG': 5, 'SMA': 5,
    'DAV': 6, 'OTL': 6, 'SAM': 6
}

# weigh by city populations

populations = {
    'GVE': 484736, 'CDF': 40143, 'NEU':  53778, 'SIO': 45932,
    'BER': 1017483, 'BAS': 191817, 'LUZ': 398762, 'STG': 499065, 'SMA': 1466424,
    'DAV': 11109, 'OTL': 63688, 'SAM': 3014 
}

data_columns = ['fu3010d1', 'tre200dx', 'tre200dn', 'tre200d0',
                'rka150d0', 'ure200dx', 'ure200dn', 'ure200d0', 
                'sre000d0', 'fu3010d0']

# Assigning region
df['region'] = df['stn'].apply(lambda x: stn_dict.get(x, np.nan))

# Calculate weights
regional_populations = {}
for city, pop in populations.items():
    region = stn_dict[city]
    regional_populations[region] = regional_populations.get(region, 0) + pop

weights = {region: populations[city] / regional_populations[region] for city, region in stn_dict.items()}

# Convert the time column to datetime
df['time'] = pd.to_datetime(df['time'])

# Mapping weights to the 'region' values
df['weight'] = df['region'].map(weights)

# Apply weights
for column in data_columns:
    df[f'weighted_{column}'] = df[column] * df['weight']

# Group by region and week, then calculate the weighted average
regional_data = df.groupby(['region', pd.Grouper(key='time', freq='W')])[[f'weighted_{column}' for column in data_columns]].mean()

# Reset index if you want 'region' and 'time' as columns
regional_data.reset_index(inplace=True)


In [8]:
regional_data

,region,time,weighted_fu3010d1,weighted_tre200dx,weighted_tre200dn,weighted_tre200d0,weighted_rka150d0,weighted_ure200dx,weighted_ure200dn,weighted_ure200d0,weighted_sre000d0,weighted_fu3010d0
0,1,2013-01-06,1.647899,0.411822,-0.091924,0.175882,0.056687,6.932636,4.898960,6.138408,10.859342,0.339508
1,1,2013-01-13,1.562190,0.296522,-0.085358,0.092712,0.101642,6.920073,5.118879,6.170494,7.894997,0.383982
2,1,2013-01-20,2.084584,0.042023,-0.361657,-0.146029,0.129745,6.608317,4.563918,5.652040,6.566032,0.486674
3,1,2013-01-27,2.163376,0.157322,-0.408932,-0.100592,0.039921,6.653754,4.298650,5.642585,11.908156,0.534475
4,1,2013-02-03,3.270409,0.499281,0.036770,0.279713,0.501907,6.848897,4.724391,5.952239,9.381547,0.847806
...,...,...,...,...,...,...,...,...,...,...,...,...
3259,6,2023-05-07,1.392058,0.636174,0.197732,0.403949,0.189432,3.607324,1.620409,2.702773,16.294474,0.300472
3260,6,2023-05-14,1.281202,0.498204,0.220789,0.333858,0.248457,3.687930,2.306939,3.135312,5.330658,0.316150
3261,6,2023-05-21,1.420648,0.529192,0.224847,0.367797,0.049802,3.497207,1.945782,2.770651,5.242121,0.378126
3262,6,2023-05-28,1.315879,0.733749,0.279629,0.500787,0.251777,3.620974,1.664678,2.764195,14.119787,0.264504
